In [2]:
import bs4 as bs
import pickle
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import fix_yahoo_finance

import requests

/home/ian/.virtualenvs/ml_pytorch/lib/python3.6/site-packages/fix_yahoo_finance/__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and will be removed in future versions.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


In [3]:
def save_SP500():
    resp = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs.BeautifulSoup(resp.text,"html5lib")
    table = soup.find("table", {"class": "wikitable sortable"})
    tickers = []
    for row in table.findAll("tr")[1:]:
        ticker = row.findAll("td")[0].text
        tickers.append(ticker)
        
    with open("S&P_500tickers.pkl", "wb") as f:
        pickle.dump(tickers, f)
        
    return tickers

#save_SP500()

In [4]:
def get_data_from_yahoo(reload_data=False):
    if reload_data:
        tickers = save_SP500()
    else:
        with open("S&P_500tickers.pkl", "rb") as f:
            tickers = pickle.load(f)
            
    if not os.path.exists("stock_prices/"):
        os.makedirs("stock_prices/")
        
    start = dt.datetime(2010, 1, 1)
    end = dt.datetime(2017, 7, 15)
    
    for ticker in tickers:
        ticker_fname = "stock_prices/{}_data.csv".format(ticker)
        if not os.path.exists(ticker_fname):
            try:
                df = web.get_data_yahoo(ticker.replace('.', '-'), start=start, end=end)
                df.to_csv(ticker_fname)
            except:
                print("Failed to get data for ticker: {0}".format(ticker))
        else:
            pass
            #print("Found data file for {0}".format(ticker))
        
get_data_from_yahoo()

In [39]:
def compile_data(ignore, csv_folder="stock_prices/", 
                 ticker_names="S&P_500tickers.pkl"):
    
    with open(ticker_names, "rb") as f:
        tickers = pickle.load(f)

    compiled_df = pd.DataFrame()

    for i, ticker in enumerate(tickers):
        
        # Sometimes when scraping we can't get all the data. Can either
        # manually scrape it later, or ignore it like we're doing now
        if ticker in ignore:
            continue
  
        df = pd.read_csv(csv_folder + "{0}_data.csv".format(ticker))
    
        # Index all the data via the dates
        df.set_index("Date", inplace=True)
        df.rename(columns = {"Adj Close": ticker}, inplace=True)
        df.drop(["Open", "High", "Low", "Close", "Volume"], 1, inplace=True)
        
        if compiled_df.empty:
            compiled_df = df
        else:
            compiled_df = compiled_df.join(df, how="outer")

        if i % 50 == 0:
            print(i)
#     print(compiled_df.head())
    
    compiled_df.to_csv("S&P500_joined_adj-close.csv")

    
ignore = [
    "HLT", "LVLT", "STI", "WU", "WRK", "WY",  "WHR", "WFM", "WMB", "WLTW", 
    "WYN", "WYNN", "XEL", "XRX", "XLNX", "XL", "XYL", "YUM", "ZBH", "ZION", 
    "ZTS"
]
compile_data(ignore)

0
50
100
150
200
250
300
350
400
450
                  MMM        ABT  ABBV        ACN       ATVI        AYI  \
Date                                                                      
2010-01-04  68.747681  21.297834   NaN  35.391899  10.328080  34.313091   
2010-01-05  68.317101  21.125759   NaN  35.610626  10.346360  34.598793   
2010-01-06  69.285927  21.243080   NaN  35.989201  10.291521  35.094002   
2010-01-07  69.335602  21.419062   NaN  35.955547  10.044743  36.189209   
2010-01-08  69.824188  21.528568   NaN  35.812534   9.962484  36.322529   

                 ADBE   AMD        AAP        AES    ...            VMC  \
Date                                                 ...                  
2010-01-04  37.090000  9.70  39.580647  11.940555    ...      49.414333   
2010-01-05  37.700001  9.71  39.345402  11.818265    ...      49.601749   
2010-01-06  37.619999  9.57  39.688480  11.695979    ...      50.463909   
2010-01-07  36.889999  9.47  39.678669  11.704714    ...      